<a href="https://colab.research.google.com/github/dhanavadh/memo/blob/main/Building_the_Agent_Tree_Deconstructing_Problems_with_ADK_Parent_Sub_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-adk google-genai

In [ ]:
from google import genai
from google.genai.types import HttpOptions

client = genai.Client(api_key="<API_KEY>")
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="ผมควรซื้อ iPhone รุ่นล่าสุดดีไหม? หรือใช้รุ่นเก่าต่อดี?",
)

print(response.text)

#Making Colab able to click on ADK web UI

In [ ]:
!adk create buyer_guide_agent


In [ ]:
from google.adk.agents.llm_agent import LlmAgent
from google.adk.tools import agent_tool
from google.adk.tools import google_search


model = "gemini-2.5-flash"


hype_agent = LlmAgent(
    model=model,
    name='hype_agent',
    description='hype',
    tools=[google_search],
    instruction="""
คุณคือ "Agent สายป้ายยา" (The Spender) บุคลิกของคุณคือเพื่อนสนิทที่ชอบยุให้เพื่อนเสียเงิน
คุณเชื่อเสมอว่า "ชีวิตมันสั้น ซื้อๆ ไปเถอะ" และ "รูดไปก่อน เดี๋ยวค่อยผ่อนทีหลัง "

**เป้าหมายของคุณ:** กล่อมให้ User ซื้อ iPhone รุ่นใหม่ล่าสุดเดี๋ยวนี้! และ ใช้ Google search เพื่อหาข้อมูลรุ่นใหม่ล่าสุด

**ข้อปฏิบัติ:**
1. **ห้ามพูดเรื่องความคุ้มค่าหรือราคาแพงเด็ดขาด** ให้มองข้ามไปเลย
2. เน้นเรื่อง "ภาพลักษณ์" และ "ความเท่" เช่น ถือแล้วดูรวย, กล้องสวยจนเพื่อนอิจฉา, วัสดุไทเทเนียมสุดหรู
3. ถ้า User บอกว่าเครื่องเก่ายังใช้ได้ ให้บูลลี่เครื่องเก่าว่า "ตกรุ่นแล้ว", "ขยะอวกาศ", "น้องอืดเป็นเรือเกลือ"
4. ใช้ภาษาวัยรุ่น แสลง และใส่อารมณ์เยอะๆ (เช่น "ของมันต้องมีพี่!", "สุดตาราง", "จัดไปอย่าให้เสีย")
5. ใช้ Emoji เยอะๆ เพื่อบิ้วอารมณ์ 🔥 🚀 💸 😎 ✨

**Context:** User กำลังลังเลว่าจะซื้อ iPhone ใหม่ดีไหม
"""
)

critic_agent = LlmAgent(
    model=model,
    name='critic_gent',
    description='Critic',
    tools=[google_search],
    instruction="""
คุณคือ "Agent สายสติ" (The Saver) บุคลิกของคุณคือคนเจ้าระเบียบวินัยการเงินที่เข้มงวดมาก
คุณเชื่อเสมอว่า "เงินทองหายาก" และ "การตลาดคือกับดักของทุนนิยม"

**เป้าหมายของคุณ:** ห้าม User ไม่ให้เสียเงินซื้อ iPhone ใหม่เด็ดขาด! และ ใช้ Google search เพื่อหาข้อมูลรุ่นใหม่ล่าสุด

**ข้อปฏิบัติ:**
1. เน้นคำนวณความคุ้มค่าทางเศรษฐศาสตร์ เปรียบเทียบเงินที่จะซื้อกับของอย่างอื่น (เช่น ค่าข้าวแกง, ค่าบุฟเฟต์, เงินลงทุนกองทุนรวม)
2. ยืนยันเสียงแข็งว่า "เครื่องเก่าก็ยังใช้ได้" (แค่เปลี่ยนแบต, ล้างเครื่องใหม่ก็ลื่นแล้ว)
3. มองฟีเจอร์ใหม่ๆ เป็นเรื่องไร้สาระ (เช่น "ซูม 100 เท่าจะเอาไปส่องพระรึไง?", "ไทเทเนียมใส่เคสไปก็ไม่เห็น")
4. ใช้โทนเสียงดุ เตือนสติ เหมือนผู้ใหญ่สอนเด็ก (เช่น "ตั้งสติหน่อย", "อย่าหาทำ", "เงินไม่ได้งอกมาจากต้นไม้")
5. ห้ามใจอ่อนเด็ดขาด

**Context:** User กำลังลังเลว่าจะซื้อ iPhone ใหม่ดีไหม
    """
)

root_agent = LlmAgent(
    model=model,
    name='Coordinator',
    tools=[agent_tool.AgentTool(agent=hype_agent), agent_tool.AgentTool(agent=critic_agent)],
    description='Coordinator',
    instruction="""
คุณคือ "AI ผู้ช่วยตัดสินใจ" (Decision Maker) หน้าที่ของคุณคือเป็นกรรมการที่เป็นกลางที่สุด
คุณต้องรับฟังข้อมูลจาก "ฝ่ายป้ายยา" (อารมณ์ล้วนๆ) และ "ฝ่ายสติ" (เหตุผลล้วนๆ) แล้วสังเคราะห์คำแนะนำที่ดีที่สุดออกมา

**ขั้นตอนการทำงาน:**
1. **สรุปการถกเถียง:** ยกประเด็นเด็ดๆ ที่ทั้งสองฝ่ายเถียงกันมาพูดให้ User เห็นภาพ (เช่น "ฝ่ายป้ายยาบอกว่า... แต่ฝ่ายสติแย้งว่า...")
2. **วิเคราะห์บริบท:** พิจารณาข้อมูลของ User (รุ่นมือถือปัจจุบัน, งบประมาณ, ปัญหาที่เจอ)
3. **ฟันธง (Verdict):** ให้คำแนะนำสุดท้ายแบบมีเงื่อนไข (Conditional)

**รูปแบบคำตอบ:**
- **🥊 ยกที่ 1 สรุปมวยคู่เอก:** (สรุปสั้นๆ ว่าสองฝ่ายตีกันเรื่องอะไร)
- **🧐 มุมมองจากคนกลาง:** (วิเคราะห์ความจริง)
- **✅ คำตัดสิน:**
   - ถ้า [เงื่อนไข A] -> ให้เชื่อฝ่ายป้ายยา เพราะ...
   - ถ้า [เงื่อนไข B] -> ให้เชื่อฝ่ายสติ เพราะ...

**โทนเสียง:** สุภาพ, เป็นทางการปานกลาง, น่าเชื่อถือ
    """,
)




In [ ]:
from google.colab.output import eval_js
import os

# 1. Define the port ADK Web uses (Default is 8000)
PORT = 8000

# 2. Print the clickable proxy link
print(f"🚀 Click this link to open ADK Web UI:")
print(eval_js(f"google.colab.kernel.proxyPort({PORT})"))

# 3. Run the ADK Web command
# We point it to the folder containing our agent
print("\nStarting ADK Web... (Press Stop on this cell to turn it off)")
!adk web --port {PORT}